In [32]:
# pip install sentence_transformers

In [33]:
from tensorflow.keras.applications.resnet50 import ResNet50,preprocess_input as resnet50_preprocess
from tensorflow.keras.applications.xception import Xception, preprocess_input as xception_preprocess
from tensorflow.keras.applications.vgg19 import VGG19, preprocess_input as vgg19_preprocess
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input as inception_v3_preprocess
from tensorflow.keras.applications.densenet import DenseNet121, preprocess_input as densenet_preprocess
from tensorflow.keras.preprocessing import image
import numpy as np
import pandas as pd
import cv2
from tqdm.auto import tqdm
from sentence_transformers import SentenceTransformer, util
from PIL import Image
import os
import tqdm

In [34]:
resnet50 = ResNet50(include_top=False, weights='imagenet', pooling='avg')

xception = Xception(include_top=False, weights='imagenet', pooling='avg')

vgg19 = VGG19(include_top=False, weights='imagenet', pooling='avg')

inceptionv3 = InceptionV3(include_top=False, weights='imagenet', pooling='avg')

densenet = DenseNet121(include_top=False, weights='imagenet', pooling='avg')

clip = SentenceTransformer('clip-ViT-B-32')


ftfy or spacy is not installed using BERT BasicTokenizer instead of ftfy.


In [35]:
def return_image_embedding(model,preprocess_input,img_path):
    if model != clip:
        img = image.load_img(img_path, target_size=(224, 224))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
        preds = model.predict(x)
        curr_df = pd.DataFrame(preds[0])
        curr_df = list(curr_df[0])
    else:
        curr_df = list(clip.encode(Image.open(img_path)))
    return curr_df

In [36]:
def files(path):
    for file in os.listdir(path):
        if os.path.isfile(os.path.join(path, file)):
            yield file

In [37]:
'''
img_path = train_img_path + "/" + img_list[0]
model = densenet
return_image_embedding(model,img_path)
'''

'\nimg_path = train_img_path + "/" + img_list[0]\nmodel = densenet\nreturn_image_embedding(model,img_path)\n'

In [38]:
def get_img_encoding_file(train_img_path,count_img,model,model_name,preprocess_input):
    i = 1
    file_num = 0
    list_img = []
    list_encodings = []


    for img in tqdm.tqdm(list(files(path = train_img_path))):

        img_path = train_img_path + "/" + img
        list_img.append(int(img[:-4]))
        list_encodings.append(return_image_embedding(model,preprocess_input,img_path))

        if (i % count_img) == 0:
            df = pd.DataFrame(list_encodings)
            df['img'] = list_img
            df.columns = df.columns.map(str)

            file_name = f"./img_encodings/{model_name}/train_img_encodings_" + f"{model_name}" + "_" + f"{file_num}" + ".parquet"

            df.to_parquet(file_name)  

            file_num = file_num + 1
            i = 1

            list_img = []
            list_encodings = []

            break

        else:

            i = i + 1

In [39]:
train_img_path = "./coco/images/train2017"
count_img = 1000

In [40]:
'''model, model_name, preprocess_input = resnet50, 'resnet50', resnet50_preprocess
get_img_encoding_file(train_img_path,count_img,model,model_name,preprocess_input)

model, model_name, preprocess_input = xception, 'xception', xception_preprocess
get_img_encoding_file(train_img_path,count_img,model,model_name,preprocess_input)

model, model_name, preprocess_input = vgg19, 'vgg19', vgg19_preprocess
get_img_encoding_file(train_img_path,count_img,model,model_name,preprocess_input)'''

model, model_name, preprocess_input = inceptionv3, 'inceptionv3', inception_v3_preprocess
get_img_encoding_file(train_img_path,count_img,model,model_name,preprocess_input)

model, model_name, preprocess_input = densenet, 'densenet', densenet_preprocess
get_img_encoding_file(train_img_path,count_img,model,model_name,preprocess_input)

model, model_name, preprocess_input = clip, 'clip', None
get_img_encoding_file(train_img_path,count_img,model,model_name,preprocess_input)

  1%|          | 999/118287 [02:26<4:46:31,  6.82it/s]


In [41]:
'''
restored_df = pd.read_parquet(path = './img_encodings/densenet/train_img_encodings_densenet_0.parquet')
restored_df
'''

"\nrestored_df = pd.read_parquet(path = './img_encodings/densenet/train_img_encodings_densenet_0.parquet')\nrestored_df\n"

In [44]:
restored_df = pd.read_parquet(path = './img_encodings/densenet/train_img_encodings_densenet_0.parquet')
restored_df

,0,1,2,3,4,5,6,7,8,9,...,1015,1016,1017,1018,1019,1020,1021,1022,1023,img
0,0.000317,0.003885,0.001279,0.001887,0.127604,0.291581,0.000696,0.003635,0.200148,0.000178,...,0.082467,1.308029,0.056614,1.188327,0.071659,2.070289,0.204783,0.899992,0.278192,428212
1,0.000487,0.003782,0.003422,0.001796,0.134881,0.193900,0.000430,0.002760,0.296177,0.000624,...,0.082419,0.456498,0.188658,0.453517,0.117408,0.369999,2.371818,0.503274,0.615941,536575
2,0.000876,0.003300,0.004119,0.001517,0.093650,0.165375,0.000953,0.003472,0.125723,0.000268,...,0.236243,4.711343,0.025890,4.004834,0.379334,1.139168,0.271448,0.452405,2.146024,1381
3,0.000256,0.004057,0.003010,0.003504,0.068183,0.748410,0.000502,0.004288,0.223687,0.000294,...,2.852961,0.260756,0.018077,0.437646,0.043522,0.015254,1.435961,0.109091,0.350480,196462
4,0.000371,0.002458,0.002845,0.003332,0.029049,0.708947,0.000702,0.003799,0.107105,0.000251,...,0.241177,0.923990,2.336812,0.878507,0.169254,0.072328,1.311640,0.053748,1.800487,26560
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.000566,0.005355,0.004124,0.001846,0.131575,0.786337,0.000415,0.005557,0.113466,0.000544,...,4.385460,2.095279,0.117258,0.585173,0.786342,3.052215,2.173588,3.133745,0.843007,507317
996,0.000331,0.004559,0.002595,0.001332,0.164859,0.121708,0.000474,0.002244,0.394074,0.000163,...,0.177247,1.592146,0.050273,0.715401,0.036046,1.361826,0.966311,3.263401,1.140627,348107
997,0.000378,0.008314,0.005174,0.002604,0.107208,0.404990,0.000840,0.000875,0.334024,0.000394,...,1.449771,1.644168,0.834730,1.017821,2.329928,0.880278,2.897640,0.939193,4.650729,458093
998,0.000457,0.006556,0.003206,0.001753,0.216390,0.495862,0.000452,0.003044,0.259392,0.000592,...,2.294128,0.432358,0.066635,0.177963,0.202715,1.506929,0.689878,0.553761,0.277268,449781


In [45]:
restored_df = pd.read_parquet(path = './img_encodings/inceptionv3/train_img_encodings_inceptionv3_0.parquet')
restored_df

,0,1,2,3,4,5,6,7,8,9,...,2039,2040,2041,2042,2043,2044,2045,2046,2047,img
0,0.758524,0.894603,0.159431,0.454656,0.453297,0.645562,0.173162,0.085253,0.500184,0.231295,...,0.818776,0.073240,0.000000,0.127824,0.140322,0.026169,0.185832,0.124089,0.000000,428212
1,0.002250,0.704307,1.385209,0.839032,0.331914,0.542251,0.055525,1.585009,0.126058,2.439339,...,0.347736,1.070790,0.787443,0.285869,0.266248,0.126200,0.000000,0.756729,0.093079,536575
2,0.312276,0.068961,0.558668,0.818934,0.294434,0.439790,0.698596,0.688306,0.219206,0.247219,...,0.218582,0.288314,1.212026,0.320030,0.485498,1.029975,0.427033,0.529195,0.164694,1381
3,0.151327,1.665143,1.501557,0.390326,0.671895,1.123622,1.678515,0.027948,0.083940,0.709190,...,1.969638,0.407883,0.574135,0.000000,0.123455,0.191167,0.281108,0.060120,1.408219,196462
4,0.356685,0.038909,1.661103,0.490681,0.030910,0.000000,0.008511,1.553757,0.268925,0.026519,...,0.000000,0.266766,2.026506,0.207744,0.015879,0.483039,1.390826,1.811844,0.395662,26560
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,1.041759,0.352495,0.804412,0.404643,0.190981,0.219685,1.218701,0.789274,0.048615,0.148554,...,0.742460,0.380717,1.386635,0.230085,2.080994,0.168398,0.031457,0.221481,0.739263,507317
996,0.200584,0.937748,0.146352,0.259076,1.147876,0.195448,0.195437,0.307904,0.313988,0.423231,...,0.016726,0.839510,0.676539,0.069720,0.126849,0.442622,0.124748,0.301242,1.269791,348107
997,0.147867,0.659754,0.383781,1.125740,0.062759,0.202742,0.955441,0.204531,0.615721,0.294295,...,0.000000,0.052201,0.000000,0.000000,0.000000,0.682523,0.315987,0.002111,1.758849,458093
998,0.804395,0.183762,2.893216,0.187461,0.021398,0.177156,0.045379,0.551443,0.333270,0.353644,...,0.926706,1.587790,0.970357,0.117892,1.747586,2.138747,0.240231,0.404455,1.313368,449781


: 